## 1. This is the first part.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
import xlrd

In [3]:
CLIENT_ID = "PRJWUKQILCPYUNAKTVZ32C4OVZ0M0BFLEO0FPBLLIA02CT1H" # your Foursquare ID
CLIENT_SECRET = 'AMYGYHZMISWJEKHGICR05SJHKBSCZDZJODZ0QPI2ENUHMYE1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [9]:
df_t = pd.read_excel(r"C:\Users\55016\Desktop\Toronto.xlsx")
df_t.drop(df_t.index[df_t['Borough']=='Not assigned'].values,inplace=True)
df_t.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [20]:
df_t.replace({'/': ','}, regex=True,inplace=True)

dft = df_t.groupby(["Postal code","Borough"],as_index=False).sum()

dft.head(10)

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_t['Borough'].unique()),
        dft.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


## 2. Let's go to the second part.

In [21]:
## It seems a bit too long to wait for the response from google, so I just use the csv file provided.
df_geo = pd.read_csv(r"C:\Users\55016\Desktop\Geospatial_Coordinates.csv")

# Let's merge it into our dataset. It's easy for that they are sorted with the same method.
df = pd.concat([dft,df_geo],axis=1)
df.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [26]:
# It seems good. We just require to drop the second "Postal code" column.
df.drop("Postal Code",axis=1,inplace=True)

In [27]:
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 3. Ok, let's rush into the final part.

### Maybe we could focus on the Borough "Scarborough". It sounds beatuiful, so as the song for it.

In [30]:
## choose the data we would use
df_s = df[df["Borough"]=="Scarborough"].reset_index(drop=True)
df_s.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [33]:
## get the top 100 venues within a radius of 500 metres.

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Scar_venues = getNearbyVenues(names=df_s['Neighborhood'],
                                   latitudes=df_s['Latitude'],
                                   longitudes=df_s['Longitude']
                                  )

Scar_venues.head()

Malvern , Rouge
Rouge Hill , Port Union , Highland Creek
Guildwood , Morningside , West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park , Ionview , East Birchmount Park
Golden Mile , Clairlea , Oakridge
Cliffside , Cliffcrest , Scarborough Village West
Birch Cliff , Cliffside West
Dorset Park , Wexford Heights , Scarborough Town Centre
Wexford , Maryvale
Agincourt
Clarks Corners , Tam O'Shanter , Sullivan
Milliken , Agincourt North , Steeles East , L'Amoreaux East
Steeles West , L'Amoreaux West
Upper Rouge


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern , Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern , Rouge",43.806686,-79.194353,T Hamilton & Son Roofing Inc,43.807985,-79.198194,Construction & Landscaping
2,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood , Morningside , West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [34]:
# Let's check how many venues were returned for each neighborhood
Scar_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff , Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners , Tam O'Shanter , Sullivan",14,14,14,14,14,14
"Cliffside , Cliffcrest , Scarborough Village West",2,2,2,2,2,2
"Dorset Park , Wexford Heights , Scarborough Town Centre",6,6,6,6,6,6
"Golden Mile , Clairlea , Oakridge",10,10,10,10,10,10
"Guildwood , Morningside , West Hill",7,7,7,7,7,7
"Kennedy Park , Ionview , East Birchmount Park",5,5,5,5,5,5


In [36]:
## Analyze each neighborhood
# one hot encoding
Scar_onehot = pd.get_dummies(Scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scar_onehot['Neighborhood'] = Scar_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scar_onehot.columns[-1]] + list(Scar_onehot.columns[:-1])
Scar_onehot = Scar_onehot[fixed_columns]

Scar_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Scar_grouped = Scar_onehot.groupby('Neighborhood').mean().reset_index()
Scar_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.0000,0.0,0.0,0.00,0.000,0.000000,0.2,0.0000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.2,0.000000,0.2,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.20,0.0,0.0000,0.000000,0.000000
1,"Birch Cliff , Cliffside West",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.25,0.000,0.000000,0.0,0.0000,0.25,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.25,0.0000,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.0000,0.000000,0.000000
2,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0000,0.0,0.0,0.00,0.125,0.000000,0.0,0.0000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.125000,0.125000,0.00,0.0000,0.125,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0000,0.125000,0.000000
3,"Clarks Corners , Tam O'Shanter , Sullivan",0.0,0.000,0.000000,0.000000,0.071429,0.0,0.000000,0.0000,0.0,0.0,0.00,0.000,0.071429,0.0,0.0000,0.00,0.0,0.0,0.0,0.0,0.000000,0.071429,0.071429,0.071429,0.00,0.0000,0.000,0.0,0.0,0.000000,0.071429,0.071429,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.071429,0.0,0.000000,0.071429,0.142857,0.0,0.071429,0.000000,0.071429,0.00,0.0,0.0000,0.071429,0.000000
4,"Cliffside , Cliffcrest , Scarborough Village West",0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.00,0.000,0.000000,0.0,0.0000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.5,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0000,0.000000,0.000000
5,"Dorset Park , Wexford Heights , Scarborough To...",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.00,0.000,0.166667,0.0,0.0000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000,0.0,0.0,0.333333,0.000000,0.000000,0.00,0.0,0.166667,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0000,0.000000,0.166667
6,"Golden Mile , Clairlea , Oakridge",0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.0000,0.2,0.1,0.00,0.000,0.000000,0.0,0.0000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000,0.0,0.1,0.000000,0.100000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.1,0.000000,0.000000,0.0,0.0000,0.000000,0.1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.1,0.0000,0.000000,0.000000
7,"Guildwood , Morningside , West Hill",0.0,0.000,0.000000,0.000000,0.142857,0.0,0.142857,0.0000,0.0,0.0,0.00,0.000,0.000000,0.0,0.0000,0.00,0.0,0.0,0.0,0.0,0.142857,0.000000,0.000000,0.000000,0.00,0.0000,0.000,0.0,0.0,0.000000,0.142857,0.000000,0.00,0.0,0.000000,0.0,0.142857,0.0,

In [39]:
# Let's print each neighborhood along with the top 5 most common venues.
num_top_venues = 5

for hood in Scar_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Scar_grouped[Scar_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1               Skating Rink   0.2
2             Breakfast Spot   0.2
3                     Lounge   0.2
4             Clothing Store   0.2


----Birch Cliff , Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                  venue  freq
0      Hakka Restaurant  0.12
1  Caribbean Restaurant  0.12
2       Thai Restaurant  0.12
3                Bakery  0.12
4                  Bank  0.12


----Clarks Corners , Tam O'Shanter , Sullivan----
                 venue  freq
0          Pizza Place  0.14
1         Intersection  0.07
2  Rental Car Location  0.07
3  Fried Chicken Joint  0.07
4          Gas Station  0.07


----Cliffside , Cliffcrest , Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1  

In [52]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scar_grouped['Neighborhood']

for ind in np.arange(Scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff , Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Construction & Landscaping,Grocery Store
3,"Clarks Corners , Tam O'Shanter , Sullivan",Pizza Place,Fried Chicken Joint,Chinese Restaurant,Gas Station,Italian Restaurant,Noodle House,Intersection,Pharmacy,Rental Car Location,Bank
4,"Cliffside , Cliffcrest , Scarborough Village West",American Restaurant,Motel,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
5,"Dorset Park , Wexford Heights , Scarborough To...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Light Rail Station,Chinese Restaurant,Bank,Bar,Gas Station,Athletics & Sports,Fried Chicken Joint
6,"Golden Mile , Clairlea , Oakridge",Bakery,Bus Line,Park,Soccer Field,Ice Cream Shop,Intersection,Metro Station,Bus Station,Vietnamese Restaurant,Construction & Landscaping
7,"Guildwood , Morningside , West Hill",Mexican Restaurant,Bank,Intersection,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Vietnamese Restaurant,Gas Station,Fried Chicken Joint
8,"Kennedy Park , Ionview , East Birchmount Park",Coffee Shop,Discount Store,Department Store,Bus Station,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
9,"Malvern , Rouge",Fast Food Restaurant,Construction & Landscaping,Vietnamese Restaurant,Thai Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store


### Now we could cluster those neighborhoods.

In [41]:
# set number of clusters
kclusters = 5

Scar_grouped_clustering = Scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 4, 1, 1, 1, 1, 0])

In [51]:

Scar_merged = df_s

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scar_merged = Scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Scar_merged.head(20) 

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Construction & Landscaping,Vietnamese Restaurant,Thai Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,2.0,History Museum,Bar,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,1.0,Mexican Restaurant,Bank,Intersection,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Vietnamese Restaurant,Gas Station,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Vietnamese Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Construction & Landscaping,Grocery Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,3.0,Playground,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029,1.0,Coffee Shop,Discount Store,Department Store,Bus Station,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577,1.0,Bakery,Bus Line,Park,Soccer Field,Ice Cream Shop,Intersection,Metro Station,Bus Station,Vietnamese Restaurant,Construction & Landscaping
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476,4.0,American Restaurant,Motel,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### I don't know what happened with the last row. I finally decide to drop it.

In [63]:
Scar_merged.drop(Scar_merged.index[Scar_merged['Neighborhood']=='Upper Rouge'].values,inplace=True)

### Good. Let's visulize the map.

In [76]:
# create map
latitude = Scar_merged[["Latitude"]].mean()
longitude = Scar_merged[["Longitude"]].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

Scar_merged[["Cluster Labels"]].astype("int")

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scar_merged['Latitude'], Scar_merged['Longitude'], Scar_merged['Neighborhood'], Scar_merged['Cluster Labels'].astype("int")):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters